In [14]:
from img2vec_keras import Img2Vec
from IPython.display import Image
import glob
import os,shutil, random

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

import cv2
import logging
logger = logging.getLogger('ftpuploader')
import edge_case_selection
import augly.image as imaugs



img2vec = Img2Vec()


# Inputs

In [12]:
input_path = "/Users/akashnair/Desktop/Akash/Project/Hypergiant/Data_Labeling/dogs-vs-cats/sample_data/"
output_path = "/Users/akashnair/Desktop/Akash/Project/Hypergiant/Data_Labeling/dogs-vs-cats/"
outlier_data_percent = 10
non_outlier_data_percent = 40
augmentation_data_percent = 20
aug_technique= "noise"   # noise, crop, vflip, rotate, saturation, brightness, scale

# Embedding Creation and Outlier generation

In [6]:
list_of_dir = [ name for name in os.listdir(input_path) if os.path.isdir(os.path.join(input_path, name)) ]
image_classes = list_of_dir

if os.path.exists(output_path+"output/outliers/"):
    shutil.rmtree(output_path+"output/outliers")

for image_class in image_classes:
    image_paths = []
    image_paths.extend(glob.glob(input_path + image_class + '/*.jpg'))

    image_vectors = {}
    for image_path in image_paths:
        vector = img2vec.get_vec(image_path)
        image_vectors[image_path] = vector

    X = np.stack(list(image_vectors.values()))

    pca_50 = PCA(n_components=50)
    pca_result_50 = pca_50.fit_transform(X)
    print('Cumulative explained variation for 50 principal components: {}'.format(np.sum(pca_50.explained_variance_ratio_)))
    print(np.shape(pca_result_50))
    
    tsne = TSNE(n_components=2, verbose=1, n_iter=3000)
    tsne_result = tsne.fit_transform(pca_result_50)

    tsne_result_scaled = StandardScaler().fit_transform(tsne_result)
    # plt.scatter(tsne_result_scaled[:,0], tsne_result_scaled[:,1])

    # images = []
    # for image_path in image_paths:
    #     image = cv2.imread(image_path, 3)
    #     b,g,r = cv2.split(image)           # get b, g, r
    #     image = cv2.merge([r,g,b])         # switch it to r, g, b
    #     image = cv2.resize(image, (50,50))
    #     images.append(image)    


    # fig, ax = plt.subplots(figsize=(20,15))
    # artists = []

    # for xy, i in zip(tsne_result_scaled, images):
    #     x0, y0 = xy
    #     img = OffsetImage(i, zoom=.7)
    #     ab = AnnotationBbox(img, (x0, y0), xycoords='data', frameon=False)
    #     artists.append(ax.add_artist(ab))
    # ax.update_datalim(tsne_result_scaled)
    # ax.autoscale(enable=True, axis='both', tight=True)
    # plt.show()



    # Outlier detection
    clf = IsolationForest(random_state=123)
    preds = clf.fit_predict(tsne_result_scaled)

    images = []
    image_paths_outlier = []
    count = 0
    outlier_count = 0
    non_outlier_count = 0

    

    if not os.path.exists(output_path+"output/outliers/outlier_data/"+image_class):
        os.makedirs(output_path+"output/outliers/outlier_data/"+image_class)

    if not os.path.exists(output_path+"output/outliers/non_outlier_data/"+image_class):
        os.makedirs(output_path+"output/outliers/non_outlier_data/"+image_class)

    tsne_result_scaled_outlier = []
    for image_path in image_paths:
        if preds[count] == -1:
            image_paths_outlier.append(image_path)
 
            image = cv2.imread(image_path, 3)
            b,g,r = cv2.split(image)           # get b, g, r
            image = cv2.merge([r,g,b])         # switch it to r, g, b
            image = cv2.resize(image, (50,50))
            images.append(image)
            tsne_result_scaled_outlier.append(tsne_result_scaled[count])
            outlier_count = outlier_count + 1

            imgName =  os.path.split(image_path)[1]
            shutil.copy(image_path, output_path+"output/outliers/outlier_data/"+image_class+"/"+imgName)
        else:

            imgName =  os.path.split(image_path)[1]
            shutil.copy(image_path, output_path+"output/outliers/non_outlier_data/"+image_class+"/"+imgName)
            non_outlier_count = non_outlier_count+1
        count = count + 1

2021-12-24 02:11:02.991845: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Cumulative explained variation for 50 principal components: 0.797802746295929
(140, 50)
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 140 samples in 0.000s...
[t-SNE] Computed neighbors for 140 samples in 0.003s...
[t-SNE] Computed conditional probabilities for sample 140 / 140
[t-SNE] Mean sigma: 9.761313
[t-SNE] KL divergence after 250 iterations with early exaggeration: 65.268768
[t-SNE] KL divergence after 1000 iterations: 0.701248
Cumulative explained variation for 50 principal components: 0.8082467317581177
(126, 50)
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 126 samples in 0.000s...
[t-SNE] Computed neighbors for 126 samples in 0.004s...
[t-SNE] Computed conditional probabilities for sample 126 / 126
[t-SNE] Mean sigma: 7.709323
[t-SNE] KL divergence after 250 iterations with early exaggeration: 63.826389
[t-SNE] KL divergence after 1300 iterations: 0.684475


# Edge case selection

In [9]:
outlier_dir = output_path+"output/outliers/outlier_data/"
non_outlier_dir = output_path+"output/outliers/non_outlier_data/"

list_of_dir_outlier = [ name for name in os.listdir(outlier_dir) if os.path.isdir(os.path.join(outlier_dir, name)) ]
# list_of_dir_non_outlier = [ name for name in os.listdir(non_outlier_dir) if os.path.isdir(os.path.join(non_outlier_dir, name)) ]

# Remove existing data
if os.path.exists(output_path+"output/improved_data/"):
    shutil.rmtree(output_path+"output/improved_data")

image_classes = list_of_dir_outlier

for image_class in image_classes:

    image_class_outlier_dir = outlier_dir+image_class
    image_class_non_outlier_dir = non_outlier_dir+image_class

    # onlyfiles = next(os.walk(image_class_outlier_dir))[2]
    total_non_outlier_data = len(os.listdir(image_class_non_outlier_dir))
    total_outlier_data = len(os.listdir(image_class_outlier_dir))
    
    total_non_outlier_data_to_select = int((int(non_outlier_data_percent)/100)*total_non_outlier_data)
    total_outlier_data_to_select = int((int(outlier_data_percent)/100)*total_outlier_data)

    # print("class : ", image_class)
    # print("outlier data : ", total_outlier_data_to_select)
    # print("non_outlier data : ", total_non_outlier_data_to_select)

    improved_output_dir = output_path+"output/improved_data/"+image_class

    # Create dir for improve data if not exist
    if not os.path.exists(improved_output_dir):
        os.makedirs(improved_output_dir)


   # Copy files from outlier folder to final data folder
   # Non outlier data specific to an image class eg. "cats"
    dirpath_non_outlier = non_outlier_dir+image_class
    filenames_ņon_outlier = random.sample(os.listdir(dirpath_non_outlier), total_non_outlier_data_to_select)
    for fname in filenames_ņon_outlier:
        srcpath = os.path.join(dirpath_non_outlier, fname)
        shutil.copy(srcpath, improved_output_dir)

   # Outlier data 
    dirpath_outlier = outlier_dir+image_class
    filenames_outlier = random.sample(os.listdir(dirpath_outlier), total_outlier_data_to_select)
    for fname in filenames_outlier:
        srcpath = os.path.join(dirpath_outlier, fname)
        shutil.copy(srcpath, improved_output_dir)

# Data Augmentation

In [16]:
outlier_dir = output_path+"output/outliers/outlier_data/"

list_of_dir_outlier = [ name for name in os.listdir(outlier_dir) if os.path.isdir(os.path.join(outlier_dir, name)) ]
# list_of_dir_non_outlier = [ name for name in os.listdir(non_outlier_dir) if os.path.isdir(os.path.join(non_outlier_dir, name)) ]

image_classes = list_of_dir_outlier

for image_class in image_classes:

    image_class_outlier_dir = outlier_dir+image_class

    total_outlier_data = len(os.listdir(image_class_outlier_dir))
    
    total_outlier_data_to_select = int((int(augmentation_data_percent)/100)*total_outlier_data)

#     print("class : ", image_class)
#     print("outlier data : ", total_outlier_data_to_select)

    improved_output_dir = output_path+"output/improved_data/"+image_class

    # Create dir for improve data if not exist
    if not os.path.exists(improved_output_dir):
        os.makedirs(improved_output_dir)


    # Copy files from outlier folder to final data folder
    # Non outlier data specific to an image class eg. "cats"
    # Outlier data 
    dirpath_outlier = outlier_dir+image_class
    filenames_outlier = random.sample(os.listdir(dirpath_outlier), total_outlier_data_to_select)
    for fname in filenames_outlier:

        if(aug_technique == "noise"):
            aug_image = imaugs.random_noise(outlier_dir+image_class+"/"+fname, output_path=output_path+"output/improved_data/"+image_class+"/aug_"+fname)
        elif(aug_technique == "crop"):
            aug_image = imaugs.crop(outlier_dir+image_class+"/"+fname, output_path=output_path+"output/improved_data/"+image_class+"/aug_"+fname)
        elif(aug_technique == "vflip"):
            aug_image = imaugs.vflip(outlier_dir+image_class+"/"+fname, output_path=output_path+"output/improved_data/"+image_class+"/aug_"+fname)
        elif(aug_technique == "rotate"):
            aug_image = imaugs.rotate(outlier_dir+image_class+"/"+fname, output_path=output_path+"output/improved_data/"+image_class+"/aug_"+fname)
        elif(aug_technique == "saturation"):
            aug_image = imaugs.saturation(outlier_dir+image_class+"/"+fname, output_path=output_path+"output/improved_data/"+image_class+"/aug_"+fname)
        elif(aug_technique == "brightness"):
            aug_image = imaugs.brightness(outlier_dir+image_class+"/"+fname, output_path=output_path+"output/improved_data/"+image_class+"/aug_"+fname)
        elif(aug_technique == "scale"):
            aug_image = imaugs.scale(outlier_dir+image_class+"/"+fname, output_path=output_path+"output/improved_data/"+image_class+"/aug_"+fname)